# Projet final — Machine Learning (M2 MAS)  
## House Price Prediction (India) — Kaggle

**Objectif :** prédire le prix d’un bien immobilier en Inde (en *lacs* = 100 000 INR) à partir de caractéristiques (type d’annonce, surface, localisation, etc.).

**Source des données :** Kaggle — *House Price Prediction Challenge*  
Lien : https://www.kaggle.com/anmolkumar/house-price-prediction-challenge

> Ce notebook est écrit pour être **reproductible** : exécuter *Kernel → Restart & Run All* doit produire les mêmes résultats (même split, mêmes CV, mêmes hyperparamètres sélectionnés).


In [1]:
# =========================
# 0) Imports & configuration
# =========================
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, GridSearchCV, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.compose import TransformedTargetRegressor

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

TRAIN_PATH = "train.csv"
TEST_PATH  = "test.csv"
SUB_PATH   = "sample_submission.csv"


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# =========================
# 1) Chargement des données
# =========================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sample_sub = pd.read_csv(SUB_PATH)

print("train:", train.shape)
print("test :", test.shape)
display(train.head())


## 1. Décrire les objectifs et les données

On dispose de deux fichiers principaux :

- **train.csv** : données d'entraînement **avec** la cible `TARGET(PRICE_IN_LACS)`  
- **test.csv** : données de test **sans** la cible (à prédire)

La variable cible est un **prix en lacs**.

Dans la suite, on va :
1. explorer les variables, vérifier les valeurs manquantes, les distributions, et quelques relations simples ;
2. construire une pipeline de prétraitement (imputation + encodage) ;
3. comparer plusieurs modèles dont au moins un **modèle ensembliste** ;
4. sélectionner des hyperparamètres avec **cross-validation** ;
5. conclure avec une évaluation finale sur un jeu de test (split) + génération éventuelle d’un fichier de soumission Kaggle.


In [ ]:
# ======================================
# 2) Exploration, visualisation, statistiques
# ======================================
target_col = "TARGET(PRICE_IN_LACS)"
assert target_col in train.columns

# Séparation X / y
X = train.drop(columns=[target_col])
y = train[target_col].copy()

# Informations globales
display(X.describe(include="all").T.head(30))
print("\nValeurs manquantes (train):")
display(X.isna().mean().sort_values(ascending=False).head(20))

print("\nDistribution de la cible (y):")
display(y.describe())

# Visualisation : distribution cible
plt.figure()
plt.hist(y, bins=60)
plt.title("Distribution de la cible: TARGET(PRICE_IN_LACS)")
plt.xlabel("Prix (lacs)")
plt.ylabel("Fréquence")
plt.show()

# Visualisation : log(1+y) (souvent utile pour les prix)
plt.figure()
plt.hist(np.log1p(y), bins=60)
plt.title("Distribution de log(1 + prix)")
plt.xlabel("log(1 + prix)")
plt.ylabel("Fréquence")
plt.show()

# Quelques checks de cardinalité des variables catégorielles
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
num_cols = [c for c in X.columns if c not in cat_cols]

print("Colonnes catégorielles:", cat_cols)
print("Colonnes numériques     :", num_cols)

for c in cat_cols:
    print(f"{c}: {X[c].nunique()} valeurs distinctes")


### Petite feature engineering (simple, justifiée)

La colonne **`ADDRESS`** a une cardinalité élevée (beaucoup d’adresses uniques).  
Un encodage one-hot direct peut créer un très grand nombre de colonnes, ce qui peut ralentir l’optimisation d’hyperparamètres.

Approche simple et robuste :
- extraire une variable **`CITY`** à partir de `ADDRESS` (ce qui réduit fortement la cardinalité) ;
- **retirer `ADDRESS`**.

C’est une hypothèse raisonnable : la ville capture une grande partie de l’information géographique, sans créer des milliers de modalités.


In [ ]:
# ======================================
# Feature engineering: CITY depuis ADDRESS
# ======================================
X_fe = X.copy()
test_fe = test.copy()

def add_city(df):
    df = df.copy()
    if "ADDRESS" in df.columns:
        df["CITY"] = df["ADDRESS"].astype(str).str.split(",").str[-1].str.strip()
        df = df.drop(columns=["ADDRESS"])
    return df

X_fe = add_city(X_fe)
test_fe = add_city(test_fe)

print("Après FE:", X_fe.shape)
print("Nunique CITY:", X_fe["CITY"].nunique())

# Visualisation rapide: prix moyen par type d'annonce (POSTED_BY)
tmp = pd.concat([X_fe[["POSTED_BY"]], y], axis=1)
by_posted = tmp.groupby("POSTED_BY")[target_col].mean().sort_values(ascending=False)

plt.figure()
plt.bar(by_posted.index.astype(str), by_posted.values)
plt.title("Prix moyen par POSTED_BY")
plt.xlabel("POSTED_BY")
plt.ylabel("Prix moyen (lacs)")
plt.show()

# Top villes (fréquence)
top_cities = X_fe["CITY"].value_counts().head(15)
plt.figure(figsize=(8,4))
plt.bar(top_cities.index, top_cities.values)
plt.title("Top 15 villes (fréquence dans train)")
plt.xticks(rotation=60, ha="right")
plt.ylabel("Nombre d'observations")
plt.tight_layout()
plt.show()


## 3. Scinder le jeu de données (train/test)

On réserve une partie des données pour l’évaluation finale, afin d’estimer la performance sur des données non vues.  
On utilise un split aléatoire **reproductible** (random_state fixé).


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_fe, y, test_size=0.2, random_state=RANDOM_STATE
)

print("Train:", X_train.shape, "Valid:", X_valid.shape)


## 4. Modèles & recherche d’hyperparamètres (Cross-validation)

Exigence : **au moins 2 modèles**, dont **au moins 1 ensembliste**.

On va comparer :
- **Ridge** (baseline de régression linéaire régularisée — utile pour benchmark, même si non compté comme “modèle principal”),
- **RandomForestRegressor** (ensembliste par bagging),
- **HistGradientBoostingRegressor** (ensembliste par boosting, performant sur données tabulaires).

### Prétraitement (pipeline)
- colonnes numériques : imputation médiane
- colonnes catégorielles : imputation de la modalité la plus fréquente + OneHotEncoder

### Cible (prix)
Les prix sont souvent très asymétriques → on teste un apprentissage sur **log(1 + y)** via `TransformedTargetRegressor`, puis on revient automatiquement à l’échelle “prix” pour l’évaluation.


In [ ]:
# ======================================
# Prétraitement commun
# ======================================
cat_cols = X_fe.select_dtypes(include=["object"]).columns.tolist()
num_cols = [c for c in X_fe.columns if c not in cat_cols]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)

def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

def evaluate_regression(y_true, y_pred, title=""):
    print(title)
    print("MAE :", mean_absolute_error(y_true, y_pred))
    print("RMSE:", rmse(y_true, y_pred))
    print("R2  :", r2_score(y_true, y_pred))


### Choix du cross-validateur

On illustre deux validateurs :

- **KFold** (5 folds) : standard, simple et rapide  
- **RepeatedKFold** : plus robuste (répète plusieurs fois le KFold), utile quand on veut réduire la variance de l’estimation

Dans la sélection d’hyperparamètres, on utilisera surtout **KFold** (pour maîtriser le temps de calcul) puis on pourra **vérifier** le meilleur modèle avec RepeatedKFold.


In [ ]:
cv_kfold = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_rep   = RepeatedKFold(n_splits=5, n_repeats=2, random_state=RANDOM_STATE)

# Scorings (on veut minimiser l'erreur -> on utilisera neg RMSE / neg MAE)
scoring = {
    "neg_rmse": "neg_root_mean_squared_error",
    "neg_mae": "neg_mean_absolute_error",
    "r2": "r2",
}


### Grilles d’hyperparamètres

Les grilles ci-dessous sont volontairement **raisonnables** (temps de calcul).  
Vous pouvez les élargir si vous avez du temps CPU.

> Astuce : sur Kaggle, le split train/valid interne n’est pas visible ; ici, on suit la consigne M2 : **CV sur train**, puis **évaluation finale** sur un hold-out (valid).


In [ ]:
# ======================================
# 4.a) Ridge (baseline)
# ======================================
ridge = Ridge(random_state=RANDOM_STATE)

ridge_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("regressor", TransformedTargetRegressor(
        regressor=ridge,
        func=np.log1p,
        inverse_func=np.expm1
    ))
])

ridge_grid = {
    "regressor__regressor__alpha": [0.1, 1.0, 10.0, 50.0],
}

# ======================================
# 4.b) RandomForest (ensembliste)
# ======================================
rf = RandomForestRegressor(
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("regressor", TransformedTargetRegressor(
        regressor=rf,
        func=np.log1p,
        inverse_func=np.expm1
    ))
])

rf_grid = {
    "regressor__regressor__n_estimators": [300, 600],
    "regressor__regressor__max_depth": [None, 10, 20],
    "regressor__regressor__min_samples_split": [2, 10],
    "regressor__regressor__min_samples_leaf": [1, 5],
}

# ======================================
# 4.c) HistGradientBoosting (ensembliste)
# ======================================
hgb = HistGradientBoostingRegressor(random_state=RANDOM_STATE)

hgb_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("regressor", TransformedTargetRegressor(
        regressor=hgb,
        func=np.log1p,
        inverse_func=np.expm1
    ))
])

hgb_grid = {
    "regressor__regressor__learning_rate": [0.03, 0.06, 0.1],
    "regressor__regressor__max_depth": [None, 6, 10],
    "regressor__regressor__max_iter": [400, 800],
    "regressor__regressor__l2_regularization": [0.0, 0.1, 1.0],
}


In [ ]:
# ======================================
# 4.d) GridSearchCV (CV sur le train)
# ======================================
def run_gridsearch(name, model, grid, cv):
    print("\n" + "="*80)
    print(f"GridSearch — {name}")
    print("="*80)

    gs = GridSearchCV(
        estimator=model,
        param_grid=grid,
        cv=cv,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1,
        verbose=1
    )
    gs.fit(X_train, y_train)
    print("Best CV RMSE:", -gs.best_score_)
    print("Best params :", gs.best_params_)
    return gs

gs_ridge = run_gridsearch("Ridge (baseline)", ridge_model, ridge_grid, cv_kfold)
gs_rf    = run_gridsearch("RandomForest", rf_model, rf_grid, cv_kfold)
gs_hgb   = run_gridsearch("HistGradientBoosting", hgb_model, hgb_grid, cv_kfold)


In [ ]:
# ======================================
# Comparaison sur le hold-out (valid)
# ======================================
best_models = {
    "Ridge": gs_ridge.best_estimator_,
    "RandomForest": gs_rf.best_estimator_,
    "HistGradientBoosting": gs_hgb.best_estimator_,
}

valid_scores = {}
for name, model in best_models.items():
    pred = model.predict(X_valid)
    valid_scores[name] = rmse(y_valid, pred)
    evaluate_regression(y_valid, pred, title=f"[VALID] {name}")
    print()

pd.Series(valid_scores).sort_values()


### Vérification plus robuste (optionnelle)

On prend le meilleur modèle selon le hold-out et on estime sa performance avec un **RepeatedKFold** (plus stable).


In [ ]:
best_name = min(valid_scores, key=valid_scores.get)
best_model = best_models[best_name]
print("Meilleur modèle (valid RMSE):", best_name, "RMSE =", valid_scores[best_name])

cv_results = cross_validate(
    best_model,
    X_train,
    y_train,
    cv=cv_rep,
    scoring=scoring,
    n_jobs=-1,
    return_train_score=False
)

# On convertit en métriques positives (car neg_* dans sklearn)
summary = pd.DataFrame({
    "RMSE": -cv_results["test_neg_rmse"],
    "MAE" : -cv_results["test_neg_mae"],
    "R2"  :  cv_results["test_r2"]
})
display(summary.describe())


## 5. Conclusion : évaluation finale + discussion

On entraîne le **meilleur modèle** sur `X_train` puis on l’évalue sur `X_valid`.  
On discute :
- la qualité des prédictions (MAE/RMSE/R²),
- l’écart train vs valid (surapprentissage potentiel),
- l’intérêt de la transformation `log1p` de la cible pour stabiliser la variance.

*(Si c’était une classification on aurait une matrice de confusion ; ici c’est une régression.)*


In [ ]:
# =========================
# Évaluation finale
# =========================
best_model.fit(X_train, y_train)
pred_valid = best_model.predict(X_valid)

evaluate_regression(y_valid, pred_valid, title=f"[FINAL VALID] {best_name}")

# Visualisation: y_true vs y_pred
plt.figure(figsize=(5,5))
plt.scatter(y_valid, pred_valid, s=8, alpha=0.5)
plt.title("Validation: y_true vs y_pred")
plt.xlabel("Prix réel (lacs)")
plt.ylabel("Prix prédit (lacs)")
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()])
plt.tight_layout()
plt.show()

# Résidus
residuals = y_valid - pred_valid
plt.figure()
plt.hist(residuals, bins=60)
plt.title("Histogramme des résidus (y_true - y_pred)")
plt.xlabel("Résidu (lacs)")
plt.ylabel("Fréquence")
plt.show()


## (Option) Prédire sur `test.csv` + créer une soumission Kaggle

Même si la consigne M2 n’impose pas Kaggle, c’est pratique de produire un fichier final.  
On s’aligne sur le format de `sample_submission.csv`.


In [ ]:
# ======================================
# Entraîner sur tout le train et prédire test
# ======================================
best_model.fit(X_fe, y)
test_preds = best_model.predict(test_fe)

# Construction submission (le nom de colonne dans sample_submission peut varier)
display(sample_sub.head())
print(sample_sub.columns)

sub = sample_sub.copy()

# Cas standard: une colonne cible unique
target_like = [c for c in sub.columns if c.lower() != "id" and c != sub.columns[0]]
# fallback: si sample_sub contient 2 colonnes [ID, TARGET]
if len(sub.columns) == 2:
    sub.iloc[:, 1] = test_preds
else:
    # sinon on suppose que la 2ème colonne est la cible
    sub.iloc[:, 1] = test_preds

SUBMISSION_PATH = "submission_best_model.csv"
sub.to_csv(SUBMISSION_PATH, index=False)
print("Submission sauvegardée:", SUBMISSION_PATH)
display(sub.head())


---

## Notes / pistes d’amélioration (si vous voulez aller plus loin)

- Tester des modèles de boosting plus avancés (XGBoost / LightGBM / CatBoost) si autorisés.
- Ajouter des features géographiques : interactions LATITUDE/LONGITUDE, clustering de zones, distance à des centres urbains.
- Ajouter des transformations sur `SQUARE_FT` ou détecter/traiter d’éventuels outliers.
- Feature engineering plus riche sur `ADDRESS` (ex: quartier + ville, hashing, embeddings texte).

L’essentiel pour la note : **démarche claire, reproductible, et discussion cohérente** des résultats.
